In [29]:
import os
import pandas as pd
import scanpy
import pandas as pd
import anndata
import os
import hdf5plugin
import os
import pickle as pkl

import pandas as pd

from pydeseq2.dds import DeseqDataSet
from pydeseq2.default_inference import DefaultInference
from pydeseq2.ds import DeseqStats
import pybiomart
import mygene
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd
import os
import requests
import json


deg_result_path = "/home/hb/python/lof/deg/norman"
save_result_path = '/home/hb/python/lof/benchmark/sigCOM_lincs'

In [6]:

print('현재까지 계산 완료된 DEG 데이터 개수:', len([x for x in os.listdir(deg_result_path) if '.csv' in x]))
print('현재까지 검증 완료된 sigcom 결과 개수:', len([x for x in os.listdir(save_result_path) if 'csv' in x]))

path = "/home/hb/python/lof/deg/norman/norman_for_deg_processing.h5ad"
adata = anndata.read_h5ad(path)
need_to_eval = list(set(adata.obs.condition.unique()) - set([x.split('_K562')[1].split('.csv')[0] for x in os.listdir(save_result_path) if 'csv' in x]) - set(['control']))

현재까지 계산 완료된 DEG 데이터 개수: 236
현재까지 검증 완료된 sigcom 결과 개수: 46


In [7]:
need_to_eval

['FOXL2_HOXB9',
 'FOXA3_FOXA1',
 'ZBTB10_DLX2',
 'HNF4A',
 'ZBTB10_PTPN12',
 'UBASH3A',
 'POU3F2_CBFA2T3',
 'ETS2_MAPK1',
 'FEV_ISL2',
 'KIF18B_KIF2C',
 'BPGM_SAMD1',
 'CEBPE_SPI1',
 'MAPK1_IKZF3',
 'PRTG',
 'IGDCC3_ZBTB25',
 'SAMD1',
 'IKZF3',
 'HES7',
 'STIL',
 'ZBTB1',
 'FOXA1_FOXL2',
 'IRF1_SET',
 'LYL1_IER5L',
 'ZBTB10_SNAI1',
 'PRDM1_CBFA2T3',
 'PTPN13',
 'CEBPE_CNN1',
 'MAPK1_TGFBR2',
 'BCORL1',
 'SGK1_TBX2',
 'DUSP9_KLF1',
 'MAP4K5',
 'FEV',
 'LYL1_CEBPB',
 'FOXL2_MEIS1',
 'FEV_CBFA2T3',
 'TGFBR2_PRTG',
 'PRDM1',
 'PTPN9',
 'BPGM_ZBTB1',
 'CEBPE_KLF1',
 'TP73',
 'BCL2L11_BAK1',
 'CBL_CNN1',
 'UBASH3B_PTPN12',
 'BCL2L11_TGFBR2',
 'CEBPE_CEBPB',
 'DUSP9',
 'HOXA13',
 'MAP2K6_IKZF3',
 'KLF1_CLDN6',
 'MAP2K3_ELMSAN1',
 'POU3F2_FOXL2',
 'FOXA3_HOXB9',
 'SNAI1_UBASH3B',
 'ARRDC3',
 'FOXF1_FOXL2',
 'CEBPA',
 'FOSB_CEBPE',
 'NIT1',
 'SET',
 'KLF1_CEBPA',
 'SLC38A2',
 'SGK1_TBX3',
 'PTPN12_PTPN9',
 'PTPN1',
 'BCL2L11',
 'DUSP9_PRTG',
 'POU3F2',
 'FOSB',
 'DUSP9_SNAI1',
 'RUNX1T1',
 'C3o

In [30]:

METADATA_API = "https://maayanlab.cloud/sigcom-lincs/metadata-api/"
DATA_API = "https://maayanlab.cloud/sigcom-lincs/data-api/api/v1/"

for CONDITION in need_to_eval:
    deg = pd.read_csv(f"/home/hb/python/lof/deg/norman/{CONDITION}.csv")
    if 'Unnamed: 0' in  deg.columns:
        del deg['Unnamed: 0']
    deg.set_index(keys=['gene_name'], inplace=True)
    deg_column = [x for x in deg.columns if 'condition' in x]
    deg = deg.sort_values(by=deg_column, ascending=False).dropna(axis=0)
    up_genes = list(deg.iloc[:700].index.values)
    down_genes = list(deg.iloc[-700:].index.values)[::-1]

    input_gene_set = {
            "up_genes": up_genes,
            "down_genes": down_genes
        }

    all_genes = input_gene_set["up_genes"] + input_gene_set["down_genes"]

    payload = {
        "filter": {
            "where": {
                "meta.symbol": {
                    "inq": all_genes
                }
            },
            "fields": ["id", "meta.symbol"]
        }
    }
    res = requests.post(METADATA_API + "entities/find", json=payload)
    entities = res.json()

    for_enrichment = {
        "up_entities": [],
        "down_entities": []
    }

    for e in entities:
        symbol = e["meta"]["symbol"]
        if symbol in input_gene_set["up_genes"]:
            for_enrichment["up_entities"].append(e["id"])
        elif symbol in input_gene_set["down_genes"]:
            for_enrichment["down_entities"].append(e["id"])
    for_enrichment['up_entities'] = for_enrichment['up_entities'][:500]
    for_enrichment['down_entities'] = for_enrichment['down_entities'][:500]
    print(len(for_enrichment['up_entities']), len(for_enrichment['down_entities']))

    payload = {
        "meta": {
            "$validator": "/dcic/signature-commons-schema/v6/meta/user_input/user_input.json",
            **for_enrichment
        },
        "type": "signature"
    }
    res = requests.post(METADATA_API + "user_input", json=payload)
    persistent_id = res.json()["id"]
    print("Access your analysis here: https://maayanlab.cloud/sigcom-lincs#/SignatureSearch/Rank/%s"%persistent_id)

    NUM_CASE = 120
    query = {
        **for_enrichment,
        "limit": NUM_CASE,
        "database": "l1000_xpr"
    }

    res = requests.post(DATA_API + "enrich/ranktwosided", json=query)
    results = res.json()

    # Optional, multiply z-down and direction-down with -1
    for i in results["results"]:
        i["z-down"] = -i["z-down"]
        i["direction-down"] = -i["direction-down"]

    results = results['results'][:NUM_CASE]
    sigids = {i['uuid']: i for i in results}

    payload = {
        "filter": {
            "where": {
                "id": {
                    "inq": list(sigids.keys())
                }
            }
        }
    }

    res = requests.post(METADATA_API + "signatures/find", json=payload)
    signatures = res.json()

    ## Merge the scores and the metadata
    for sig in signatures:
        uid = sig["id"]
        scores = sigids[uid]
        # scores.pop("uuid")
        sig["scores"] = scores

    for i in signatures:
        dict1 = i['meta']
        dict2 = i['scores']

        dict1.update(dict2)

    df_mimickers = pd.DataFrame([x['meta'] for x in signatures])
    df_mimickers = df_mimickers.sort_values(by=['z-sum'], ascending=False)
    df_mimickers.index = [f'Top-{x}' for x in range(1, len(df_mimickers)+1)]
    df_mimickers.dropna(subset=['pert_name'], inplace=True)
    df_mimickers.to_csv(f'/home/hb/python/lof/benchmark/sigCOM_lincs/Norman_K562_{CONDITION}.csv')
    print(f'{CONDITION} saved!')

500 500
Access your analysis here: https://maayanlab.cloud/sigcom-lincs#/SignatureSearch/Rank/0f26f86a-7622-5c02-92b9-527967034d8e
FOXL2_HOXB9 saved!
500 500
Access your analysis here: https://maayanlab.cloud/sigcom-lincs#/SignatureSearch/Rank/628c8fa3-3dfd-5c7e-9ece-b026221a3fb6
FOXA3_FOXA1 saved!
500 500
Access your analysis here: https://maayanlab.cloud/sigcom-lincs#/SignatureSearch/Rank/2077289b-b87f-5791-b07e-0f5e4f4f6e67
ZBTB10_DLX2 saved!
500 500
Access your analysis here: https://maayanlab.cloud/sigcom-lincs#/SignatureSearch/Rank/2f4a5e0a-ab44-5905-afdd-cea5cfda2f04
HNF4A saved!
500 500
Access your analysis here: https://maayanlab.cloud/sigcom-lincs#/SignatureSearch/Rank/387cb3fb-a553-5b57-b4fc-3515ace1073c
ZBTB10_PTPN12 saved!
500 500
Access your analysis here: https://maayanlab.cloud/sigcom-lincs#/SignatureSearch/Rank/e7df4755-c38f-5855-bf95-1127ce12cdd7
UBASH3A saved!
500 500
Access your analysis here: https://maayanlab.cloud/sigcom-lincs#/SignatureSearch/Rank/cd4fa825-0df4-